In [1]:
#!pip install pandas
#!pip install matplotlib
#!pip install seaborn
#!pip install scikit-learn

import math
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import sklearn
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.model_selection import train_test_split

In [2]:
pwd

'/Users/haroldcha/Desktop/DIGHUM101_Project/Notebooks'

In [3]:
cd "../Data/sample_child_survey/csv"

/Users/haroldcha/Desktop/DIGHUM101_Project/Data/sample_child_survey/csv


In [4]:
ls

samchild_2000.csv  samchild_2005.csv  samchild_2010.csv  samchild_2015.csv*
samchild_2001.csv  samchild_2006.csv  samchild_2011.csv  samchild_2016.csv*
samchild_2002.csv  samchild_2007.csv  samchild_2012.csv  samchild_2017.csv*
samchild_2003.csv  samchild_2008.csv  samchild_2013.csv  samchild_2018.csv*
samchild_2004.csv  samchild_2009.csv  samchild_2014.csv


In [5]:
data_name_2000s = "samchild_200{}.csv" # general format for reading data files from 2000's
data_name_2010s = "samchild_20{}.csv" # general format for reading data files from 2010's

samchild_2000s = {} # dictionary with year number of 2000s as key and respective dataframe from specified year as value
samchild_2010s = {} # dictionary with year number of 2010s " " " "

for i in range(0, 10):
    samchild_2000s["{0}".format(i)] = pd.read_csv(data_name_2000s.format(i))
for i in range(10, 19):
    samchild_2010s["{0}".format(i)] = pd.read_csv(data_name_2010s.format(i))

In [6]:
child_2000 = samchild_2000s["0"]
child_2001 = samchild_2000s["1"]
child_2002 = samchild_2000s["2"]
child_2003 = samchild_2000s["3"]
child_2004 = samchild_2000s["4"]
child_2005 = samchild_2000s["5"]
child_2006 = samchild_2000s["6"]
child_2007 = samchild_2000s["7"]
child_2008 = samchild_2000s["8"]
child_2009 = samchild_2000s["9"]

child_2010 = samchild_2010s["10"]
child_2011 = samchild_2010s["11"]
child_2012 = samchild_2010s["12"]
child_2013 = samchild_2010s["13"]
child_2014 = samchild_2010s["14"]
child_2015 = samchild_2010s["15"]
child_2016 = samchild_2010s["16"]
child_2017 = samchild_2010s["17"]
child_2018 = samchild_2010s["18"]

In [7]:
child_2000

,RECTYPE,SRVY_YR,HHX,FMX,PX,CSRESPNO,CSRELTIV,INTV_QRT,SEX,AGE_P,...,CHCSYR10,CHPXYR_C,CHERNOY2,CHCHYR,CHCHMOYR,CHCHNOY2,CHCNOYR2,CSRGYR,CSRGNOYR,CMDLONGR
0,40,2000,1,1,5,2,1,1,1,4,...,NaN,2,0,2,NaN,NaN,0,2,NaN,3
1,40,2000,2,1,3,1,1,1,1,9,...,2.0,2,1,2,NaN,NaN,2,2,NaN,1
2,40,2000,8,1,3,1,1,1,2,3,...,2.0,1,0,2,NaN,NaN,2,2,NaN,1
3,40,2000,12,1,4,2,1,1,2,17,...,NaN,2,1,2,NaN,NaN,0,2,NaN,2
4,40,2000,14,1,6,2,1,1,1,9,...,2.0,1,0,2,NaN,NaN,2,2,NaN,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13371,40,2000,72355,1,6,2,1,4,1,4,...,NaN,2,0,2,NaN,NaN,3,2,NaN,1
13372,40,2000,72357,1,4,1,1,4,2,12,...,NaN,2,0,2,NaN,NaN,0,2,NaN,3
13373,40,2000,72359,1,4,2,1,4,2,4,...,2.0,1,1,2,NaN,NaN,3,2,NaN,1
13374,40,2000,72362,1,4,3,1,4,1,6,...,NaN,2,0,2,NaN,NaN,0,2,NaN,2
